In [ ]:
#| default_exp phm2008

# PHM2008 dataset

> Download the PHM2008 dataset.

Prognosis and Health Management 2008 challenge dataset. This dataset used the Commercial Modular Aero-Propulsion System Simulation to recreate the degradation process of turbofan engines for different aircraft with varying wear and manufacturing starting under normal conditions. The training dataset consists of complete run-to-failure simulations, while the test dataset comprises sequences before failure.

[Saxena, A., Goebel, K., Simon, D.,&Eklund, N. (2008). "Damage propagation modeling for aircraft engine run-to-failure simulation". International conference on prognostics and health management.](https://ntrs.nasa.gov/api/citations/20090029214/downloads/20090029214.pdf)

In [ ]:
#| export
import os
from dataclasses import dataclass
from typing import Tuple

import pandas as pd

from datasetsforecast.utils import extract_file, download_file, Info

In [ ]:
#| export
@dataclass
class FD001:
    seasonality: int = 1
    horizon: int = 1
    freq: str = 'None'
    train_file: str = 'train_FD001.txt'
    test_file: str = 'test_FD001.txt'
    rul_file: str = 'RUL_FD001.txt'
    n_ts: int = 100
    n_test: int = 100

@dataclass
class FD002:
    seasonality: int = 1
    horizon: int = 1
    freq: str = 'None'
    train_file: str = 'train_FD002.txt'
    test_file: str = 'test_FD002.txt'
    rul_file: str = 'RUL_FD002.txt'
    n_ts: int = 260
    n_test: int = 259

@dataclass
class FD003:
    seasonality: int = 1
    horizon: int = 1
    freq: str = 'None'
    train_file: str = 'train_FD003.txt'
    test_file: str = 'test_FD003.txt'
    rul_file: str = 'RUL_FD003.txt'
    n_ts: int = 100
    n_test: int = 100

@dataclass
class FD004:
    seasonality: int = 1
    horizon: int = 8
    freq: str = 'None'
    train_file: str = 'train_FD004.txt'
    test_file: str = 'test_FD004.txt'
    rul_file: str = 'RUL_FD004.txt'
    n_ts: int = 249
    n_test: int = 248

In [ ]:
#| export
PHM2008Info = Info((FD001, FD002, FD003, FD004))

In [ ]:
#| export
@dataclass
class PHM2008:

    #source_url = 'https://forecasters.org/data/m3comp/M3C.xls'
    source_url = 'https://www.dropbox.com/s/1od45uh37tplxt7/CMAPSSData.zip?dl=1'

    @staticmethod
    def download(directory: str) -> None:
        """
        Download PHM2008 Dataset.
        
        Parameters
        ----------
        directory: str
            Directory path to download dataset.
        """
        path = f'{directory}/phm2008/'
        if not os.path.exists(path):
            download_file(path, PHM2008.source_url)
        if not os.path.isdir(f'{directory}/phm2008/CMAPSSData'):
            extract_file(f'{path}CMAPSSData.zip', path)

    @staticmethod
    def load(directory: str, group: str, clip: bool=True) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Downloads and loads M3 data.

        Parameters
        ----------
        directory: str
            Directory where data will be downloaded.
        group: str
            Group name.
            Allowed groups: 'FD001', 'FD002', 'FD003', 'FD004'.

        Returns
        -------
        df: pd.DataFrame
            Target time series with columns ['unique_id', 'ds', 'y', 'exogenous'].
        """
        def _add_remaining_useful_life(df):
            # Get the total number of cycles for each unit
            grouped_by_unit = df.groupby(by="unit_nr")
            max_cycle = grouped_by_unit["time_cycles"].max()

            # Merge the max cycle back into the original frame
            result_frame = df.merge(max_cycle.to_frame(name='max_cycle'), left_on='unit_nr', right_index=True)

            # Calculate remaining useful life for each row (piece-wise Linear)
            remaining_useful_life = result_frame["max_cycle"] - result_frame["time_cycles"]
            result_frame["RUL"] = remaining_useful_life

            # drop max_cycle as it's no longer needed
            result_frame = result_frame.drop("max_cycle", axis=1)

            return result_frame

        PHM2008.download(directory)
        
        path = f'{directory}/phm2008/CMAPSSData'
        group = PHM2008Info.get_group(group)
        
        # define column names for easy indexing
        index_names = ['unit_nr', 'time_cycles']
        setting_names = ['setting_1', 'setting_2', 'setting_3']
        sensor_names = ['s_{}'.format(i) for i in range(1, 22)]
        col_names = index_names + setting_names + sensor_names

        # read data
        train = pd.read_csv((f'{path}/{group.train_file}'), 
                            sep='\s+', header=None, names=col_names)
        test = pd.read_csv((f'{path}/{group.test_file}'), 
                        sep='\s+', header=None, names=col_names)
        y_test = pd.read_csv((f'{path}/{group.rul_file}'), 
                            sep='\s+', header=None, names=['RUL'])

        # drop non-informative features in training set
        drop_sensors = ['s_1', 's_5', 's_6', 's_10', 's_16', 's_18', 's_19']
        drop_labels = setting_names + drop_sensors
        train.drop(labels=drop_labels, axis=1, inplace=True)

        # Add piece-wise target remaining useful life
        # in the paper the MAX RUL is mentioned as 125
        train = _add_remaining_useful_life(train)
        if clip:
            train['RUL'].clip(upper=125, inplace=True)

        # Training set
        Y_train_df = train.rename(columns={'unit_nr': 'unique_id',
                                           'time_cycles': 'ds', 'RUL': 'y'})

        # drop non-informative features in testing set
        test.drop(labels=drop_labels, axis=1, inplace=True)

        # Testing set
        Y_test_df = test.rename(columns={'unit_nr': 'unique_id',
                                         'time_cycles': 'ds', 'RUL': 'y'})

        return Y_train_df, Y_test_df

In [ ]:
#| hide
for group, meta in PHM2008Info:
    Y_train_df, Y_test_df = PHM2008.load(directory='data', group=group)
    
    unique_elements = Y_train_df.groupby(['unique_id', 'ds']).size()
    unique_ts = Y_train_df.groupby('unique_id').size()
    assert (unique_elements != 1).sum() == 0, f'Duplicated records found: {group}'
    assert unique_ts.shape[0] == meta.n_ts, f'Number of time series not match: {group}'

    unique_elements = Y_test_df.groupby(['unique_id', 'ds']).size()
    unique_ts = Y_test_df.groupby('unique_id').size()
    assert (unique_elements != 1).sum() == 0, f'Duplicated records found: {group}'
    assert unique_ts.shape[0] == meta.n_test, f'Number of time series not match: {group}'